In [1]:
import pandas as pd

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,49,2024-02-07,6,114.770833,97.643750,0.556750,11.631250,24.970833,0.183083,117.083333,...,11.722449,19.816327,0.180837,112.036735,0,2,2,2,6,2024
1,51,2024-02-07,-3,114.280000,98.190000,0.545440,11.478000,25.064000,0.187060,115.480000,...,11.704082,22.422449,0.169429,110.736735,0,2,2,2,28,2024
2,51,2024-02-07,-1,121.640000,100.704000,0.542020,10.946000,28.006000,0.216260,119.768000,...,10.610000,24.884000,0.199200,122.332000,0,2,2,2,1,2024
3,48,2024-02-07,1,118.744681,98.621277,0.554532,12.061702,27.372340,0.200000,118.921277,...,10.000000,24.965306,0.255898,120.665306,0,2,2,2,10,2024
4,51,2024-02-07,-4,112.340000,101.012000,0.529360,12.518000,22.894000,0.174060,110.872000,...,11.318000,22.358000,0.226300,114.462000,0,2,2,4,27,2024
5,51,2024-02-07,3,116.880000,98.090000,0.556660,11.282000,25.766000,0.216300,119.032000,...,10.967347,25.226531,0.214449,122.208163,0,2,2,2,19,2024
6,50,2024-02-07,-3,112.591837,99.710204,0.536653,13.179592,25.065306,0.198265,112.489796,...,11.289796,23.402041,0.178918,117.987755,0,2,2,3,9,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', show=True):
    if model_name == 'LR':
        model = joblib.load('Models/TrainedModels/LR_Model.pkl')
        scaler = joblib.load('Models/Scalers/LR.bin')
    elif model_name == 'RF':
        model = joblib.load('Models/TrainedModels/RF_Model.pkl')
        scaler = joblib.load('Models/Scalers/RF.bin')
    elif model_name == 'GB':
        model = joblib.load('Models/TrainedModels/GB_Model.pkl')
        scaler = joblib.load('Models/Scalers/GB.bin')
    elif model_name == 'SVM':
        model = joblib.load('Models/TrainedModels/SVM_Model.pkl')
        scaler = joblib.load('Models/Scalers/SVM.bin')
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    predictions = model.predict(scaled_df)
    
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-07,CLE,WAS,1.160000,5.550000,1
1,2024-02-07,TOR,CHO,1.350877,3.300000,0
2,2024-02-07,ATL,BOS,6.500000,1.125000,0
3,2024-02-07,GSW,PHI,1.505051,2.640000,0
4,2024-02-07,SAS,MIA,3.750000,1.289855,0
5,2024-02-07,NOP,LAC,3.100000,1.387597,0
6,2024-02-07,DET,SAC,6.750000,1.117647,0


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-07,CLE,WAS,1.160000,5.550000,1
1,2024-02-07,TOR,CHO,1.350877,3.300000,1
2,2024-02-07,ATL,BOS,6.500000,1.125000,0
3,2024-02-07,GSW,PHI,1.505051,2.640000,0
4,2024-02-07,SAS,MIA,3.750000,1.289855,0
5,2024-02-07,NOP,LAC,3.100000,1.387597,0
6,2024-02-07,DET,SAC,6.750000,1.117647,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-07,CLE,WAS,1.160000,5.550000,1
1,2024-02-07,TOR,CHO,1.350877,3.300000,1
2,2024-02-07,ATL,BOS,6.500000,1.125000,0
3,2024-02-07,GSW,PHI,1.505051,2.640000,0
4,2024-02-07,SAS,MIA,3.750000,1.289855,0
5,2024-02-07,NOP,LAC,3.100000,1.387597,0
6,2024-02-07,DET,SAC,6.750000,1.117647,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-07,CLE,WAS,1.160000,5.550000,1
1,2024-02-07,TOR,CHO,1.350877,3.300000,0
2,2024-02-07,ATL,BOS,6.500000,1.125000,0
3,2024-02-07,GSW,PHI,1.505051,2.640000,0
4,2024-02-07,SAS,MIA,3.750000,1.289855,0
5,2024-02-07,NOP,LAC,3.100000,1.387597,0
6,2024-02-07,DET,SAC,6.750000,1.117647,0
